# PyPIPR Package Example: Pupil Series Analysis Pipeline

This example demonstrates how to analyze continuous pupillometry recordings with multiple stimuli using the PyPIPR package.

## What you'll learn:

1. **Series Data Loading**: How to load continuous pupillometry recordings
2. **Data Preprocessing**: Rate limiting, smoothing, and artifact removal for series data
3. **Series Splitting**: Extracting individual responses from continuous recordings
4. **Batch Analysis**: Processing multiple measurements efficiently
5. **Comparative Analysis**: Analyzing response consistency across trials
6. **Visualization**: Creating comprehensive series plots

## Package Structure Used:

- `pypipr.data` - Example dataset loading
- `pypipr.core` - Core data structures (PupilSeries, PupilMeasurement)
- `pypipr.analysis.fitting` - Comprehensive fitting with PupilFit
- `pypipr.preprocessing` - Data cleaning and filtering

Perfect for analyzing experimental sessions with multiple trials! 📊

# Step 1: Import Required Libraries

Import all necessary libraries and modules from the reorganized PyPIPR package.

In [ ]:
import pypipr
import matplotlib.pyplot as plt
import numpy as np

# Import specific functions from reorganized modules
from pypipr.data import load_real_series
from pypipr.analysis.fitting import PupilFit

# Step 2: Load and Explore Series Data

We'll work with real pupillometry data that contains multiple light stimuli in a continuous recording.

In [ ]:
# Load real pupillometry series data with multiple stimuli
print("Loading real pupil series data...")
series = load_real_series()

print(f"Series data: {len(series.get_time())} time points")
print(f"Time range: {series.get_time()[0]:.1f}s to {series.get_time()[-1]:.1f}s")
print(f"Number of light stimuli: {len(series.get_light_stimuli().get_times())}")

# Show stimulus timing
stimuli_times = series.get_light_stimuli().get_times()
for i, (start, end) in enumerate(stimuli_times):
    print(f"Stimulus {i+1}: {start:.1f}s - {end:.1f}s (duration: {end-start:.1f}s)")

# Step 3: Visualize Raw Series Data

Let's first look at the complete series to understand the data structure and identify any obvious artifacts.

In [ ]:
# Plot the complete series with stimulus markers
fig, ax = plt.subplots(1, 1, figsize=(16, 6))

# Plot pupil trace
series.plot(ax=ax, label="Pupil Trace", color="darkblue", alpha=0.8, linewidth=1)

# Add light stimuli markers
series.plot_light_stimulus(ax=ax, color="yellow", alpha=0.4, label="Light Stimuli")

ax.set_title("Complete Pupillometry Series with Multiple Light Stimuli")
ax.set_xlabel("Time (s)")
ax.set_ylabel("Pupil Size (mm)")
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Show basic statistics
pupil_data = series.get_size()
print(f"\nSeries Statistics:")
print(f"Mean pupil size: {np.mean(pupil_data):.3f} mm")
print(f"Standard deviation: {np.std(pupil_data):.3f} mm")
print(f"Range: {np.min(pupil_data):.3f} - {np.max(pupil_data):.3f} mm")
print(f"Recording duration: {series.get_time()[-1] - series.get_time()[0]:.1f} seconds")

# Step 4: Series Data Preprocessing Pipeline

Real pupillometry data often contains artifacts and noise. Let's apply a comprehensive preprocessing pipeline to clean the continuous series data.

In [ ]:
# Demonstrate preprocessing pipeline for series data
fig, ax = plt.subplots(4, 1, figsize=(16, 14), sharex=True)

# Step 1: Original data
series.plot(ax=ax[0], label="Raw Pupil Trace", color="darkblue", alpha=0.7)
ax[0].set_title("Step 1: Raw Series Data")
ax[0].set_ylabel("Pupil Size (mm)")
ax[0].legend()
ax[0].grid(True, alpha=0.3)

# Step 2: Rate limiting (remove artifacts)
series_step2 = series.copy()
series_step2.limit_rate_of_change(2.0)  # Remove points with >2 mm/s change
series_step2.drop_nan()
series_step2.plot(ax=ax[1], label="Rate Limited (< 2 mm/s)", color="orange", alpha=0.8)
ax[1].set_title("Step 2: Rate Limiting (Artifact Removal)")
ax[1].set_ylabel("Pupil Size (mm)")
ax[1].legend()
ax[1].grid(True, alpha=0.3)

# Step 3: Smoothing
series_step3 = series_step2.copy()
series_step3.rolling_mean(0.5)  # 0.5 second rolling mean
series_step3.plot(ax=ax[2], label="Smoothed (0.5s window)", color="purple", alpha=0.8)
ax[2].set_title("Step 3: Smoothing Filter")
ax[2].set_ylabel("Pupil Size (mm)")
ax[2].legend()
ax[2].grid(True, alpha=0.3)

# Step 4: Final with light stimuli
series_final = series_step3
series_final.plot(ax=ax[3], label="Processed Data", color="darkgreen", linewidth=2)
series_final.plot_light_stimulus(ax=ax[3], color="yellow", alpha=0.4, label="Light Stimuli")
ax[3].set_title("Step 4: Final Processed Series with Light Stimuli")
ax[3].set_xlabel("Time (s)")
ax[3].set_ylabel("Pupil Size (mm)")
ax[3].legend()
ax[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Data quality improvement:")
print(f"Data points removed by rate limiting: {len(series.get_time()) - len(series_step2.get_time())}")
print(f"Percentage of data retained: {100 * len(series_step2.get_time()) / len(series.get_time()):.1f}%")
print(f"Noise reduction (std): {np.std(series.get_size()):.3f} → {np.std(series_final.get_size()):.3f} mm")

# Step 5: Split Series into Individual Measurements

Now we'll extract individual pupil responses from the continuous series data around each light stimulus.

In [ ]:
# Split the series into individual measurements around each stimulus
print("Splitting series into individual measurements...")
pupil_measurements = series_final.split(prepulse_duration=10.0, postpulse_duration=30.0)
print(f"Created {len(pupil_measurements)} individual measurements")

# Show details about each extracted measurement
for i, measurement in enumerate(pupil_measurements):
    time_range = measurement.get_time()[-1] - measurement.get_time()[0]
    stimulus_info = measurement.get_light_stimulus()
    baseline = measurement.find_baseline(duration=5.0)
    
    print(f"\nMeasurement {i+1}:")
    print(f"  Time range: {time_range:.1f} seconds")
    print(f"  Data points: {len(measurement.get_time())}")
    print(f"  Baseline: {baseline:.3f} mm")
    print(f"  Light stimulus: {stimulus_info}")

# Step 6: Visualize Individual Measurements

Let's plot each extracted measurement to see the individual pupil responses.

In [ ]:
# Plot each individual measurement
fig, axes = plt.subplots(len(pupil_measurements), 1, figsize=(15, 4*len(pupil_measurements)), 
                        sharex=True, sharey=True)

if len(pupil_measurements) == 1:
    axes = [axes]  # Make it iterable for single subplot

colors = ['blue', 'red', 'green', 'purple', 'orange']  # Different colors for each measurement

for i, measurement in enumerate(pupil_measurements):
    ax = axes[i]
    color = colors[i % len(colors)]
    
    # Apply baseline correction for better visualization
    measurement_corrected = measurement.copy()
    baseline = measurement_corrected.find_baseline(duration=5.0)
    measurement_corrected.apply_baseline(baseline)
    
    # Plot measurement
    measurement_corrected.plot(ax=ax, label=f"Measurement {i+1} (Baseline Corrected)", 
                              color=color, alpha=0.8, linewidth=2)
    
    # Plot light stimulus
    measurement.plot_light_stimulus(ax=ax, alpha=0.4, color="yellow", label="Light Stimulus")
    
    # Add baseline reference line
    ax.axhline(y=1.0, color='gray', linestyle='--', alpha=0.5, label='Baseline')
    
    ax.set_title(f"Individual Measurement {i+1} - Pupil Light Response")
    ax.set_ylabel("Relative Pupil Size")
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Add some basic metrics as text
    pupil_data = measurement_corrected.get_size()
    min_response = np.min(pupil_data)
    max_constriction = 1.0 - min_response
    
    ax.text(0.02, 0.98, f'Max Constriction: {max_constriction:.2f}\nBaseline: {baseline:.2f} mm', 
            transform=ax.transAxes, verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

axes[-1].set_xlabel("Time (s)")
plt.tight_layout()
plt.show()

# Step 7: Comprehensive Batch Analysis

Now we'll perform comprehensive fitting analysis on all individual measurements and compare the results.

In [ ]:
# Create comprehensive fits for each measurement
print("Performing comprehensive fitting analysis on all measurements...")
fits = []
fit_params = []

for i, measurement in enumerate(pupil_measurements):
    print(f"Fitting measurement {i+1}...")
    
    # Apply baseline correction
    measurement_corrected = measurement.copy()
    baseline = measurement_corrected.find_baseline(duration=5.0)
    measurement_corrected.apply_baseline(baseline)
    
    # Create and fit comprehensive model
    fit = PupilFit.from_measurement(measurement_corrected)
    fit.fit_all()
    
    fits.append(fit)
    
    # Extract parameters
    params = fit.get_all_params()
    fit_params.append(params)
    
    print(f"  Fit completed for measurement {i+1}")

print(f"\nCompleted fitting for {len(fits)} measurements")

In [ ]:
# Visualize fits for all measurements
fig, axes = plt.subplots(len(pupil_measurements), 1, figsize=(15, 4*len(pupil_measurements)), 
                        sharex=True, sharey=True)

if len(pupil_measurements) == 1:
    axes = [axes]  # Make it iterable for single subplot

for i, (measurement, fit) in enumerate(zip(pupil_measurements, fits)):
    ax = axes[i]
    
    # Apply baseline correction
    measurement_corrected = measurement.copy()
    baseline = measurement_corrected.find_baseline(duration=5.0)
    measurement_corrected.apply_baseline(baseline)
    
    # Plot raw measurement
    measurement_corrected.plot(ax=ax, label=f"Measurement {i+1}", color="blue", alpha=0.7)
    
    # Plot light stimulus
    measurement.plot_light_stimulus(ax=ax, alpha=0.4, color="yellow", label="Light Stimulus")
    
    # Plot comprehensive fit
    time_fit = np.linspace(measurement_corrected.get_time()[0], measurement_corrected.get_time()[-1], 500)
    prediction = fit.predict(time_fit)
    ax.plot(time_fit, prediction, 'r--', linewidth=2, alpha=0.8, label="Complete Fit")
    
    ax.set_title(f"Measurement {i+1} - Data and Comprehensive Fit")
    ax.set_ylabel("Relative Pupil Size")
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Add parameter info
    params = fit_params[i]
    param_text = "\n".join([f"{k}: {v}" for k, v in list(params.items())[:3]])  # Show first 3 params
    ax.text(0.98, 0.98, param_text, transform=ax.transAxes, 
            verticalalignment='top', horizontalalignment='right',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

axes[-1].set_xlabel("Time (s)")
plt.tight_layout()
plt.show()

# Step 8: Cross-Trial Analysis and Comparison

Let's analyze the consistency and variability across all measurements in the series.

In [ ]:
# Extract key metrics from all measurements for comparison
print("=== CROSS-TRIAL ANALYSIS ===\n")

# Collect metrics from all measurements
baselines = []
max_constrictions = []
response_latencies = []

for i, measurement in enumerate(pupil_measurements):
    # Apply baseline correction
    measurement_corrected = measurement.copy()
    baseline = measurement_corrected.find_baseline(duration=5.0)
    measurement_corrected.apply_baseline(baseline)
    
    # Calculate basic metrics
    pupil_data = measurement_corrected.get_size()
    time_data = measurement_corrected.get_time()
    
    baselines.append(baseline)
    
    # Max constriction (1.0 - minimum relative size)
    min_response = np.min(pupil_data)
    max_constriction = 1.0 - min_response
    max_constrictions.append(max_constriction)
    
    # Response latency (time to reach 10% of max constriction)
    constriction_threshold = 1.0 - 0.1 * max_constriction
    latency_indices = np.where((pupil_data < constriction_threshold) & (time_data > 0))[0]
    latency = time_data[latency_indices[0]] - time_data[0] if len(latency_indices) > 0 else np.nan
    response_latencies.append(latency)
    
    print(f"Measurement {i+1}:")
    print(f"  Baseline: {baseline:.3f} mm")
    print(f"  Max Constriction: {max_constriction:.3f} (relative)")
    print(f"  Response Latency: {latency:.3f} s")
    print()

# Calculate summary statistics
print("SUMMARY STATISTICS ACROSS ALL TRIALS:")
print(f"Baseline diameter: {np.mean(baselines):.3f} ± {np.std(baselines):.3f} mm")
print(f"Max constriction: {np.mean(max_constrictions):.3f} ± {np.std(max_constrictions):.3f}")
print(f"Response latency: {np.nanmean(response_latencies):.3f} ± {np.nanstd(response_latencies):.3f} s")

# Calculate coefficient of variation (CV) for reliability assessment
cv_baseline = np.std(baselines) / np.mean(baselines) * 100
cv_constriction = np.std(max_constrictions) / np.mean(max_constrictions) * 100
cv_latency = np.nanstd(response_latencies) / np.nanmean(response_latencies) * 100

print(f"\nCOEFFICIENT OF VARIATION (reliability):")
print(f"Baseline CV: {cv_baseline:.1f}%")
print(f"Max constriction CV: {cv_constriction:.1f}%")
print(f"Response latency CV: {cv_latency:.1f}%")

In [ ]:
# Create comprehensive comparison visualizations
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: All measurements overlaid (baseline corrected)
ax1 = axes[0, 0]
colors = plt.cm.viridis(np.linspace(0, 1, len(pupil_measurements)))

for i, measurement in enumerate(pupil_measurements):
    measurement_corrected = measurement.copy()
    baseline = measurement_corrected.find_baseline(duration=5.0)
    measurement_corrected.apply_baseline(baseline)
    
    # Normalize time to start from stimulus onset
    time_normalized = measurement_corrected.get_time() - measurement_corrected.get_time()[0]
    ax1.plot(time_normalized, measurement_corrected.get_size(), 
            color=colors[i], alpha=0.7, label=f'Trial {i+1}')

# Add average response
if len(pupil_measurements) > 1:
    # Calculate average (simplified - assumes same time points)
    ax1.axhline(y=1.0, color='black', linestyle='--', alpha=0.5, label='Baseline')

ax1.set_title('All Trials Overlaid (Baseline Corrected)')
ax1.set_xlabel('Time from Trial Start (s)')
ax1.set_ylabel('Relative Pupil Size')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Metric comparison across trials
ax2 = axes[0, 1]
trial_numbers = range(1, len(pupil_measurements) + 1)

ax2_twin = ax2.twinx()
line1 = ax2.plot(trial_numbers, baselines, 'o-', color='blue', label='Baseline (mm)', markersize=8)
line2 = ax2_twin.plot(trial_numbers, max_constrictions, 's-', color='red', label='Max Constriction', markersize=8)

ax2.set_xlabel('Trial Number')
ax2.set_ylabel('Baseline Diameter (mm)', color='blue')
ax2_twin.set_ylabel('Max Constriction (relative)', color='red')
ax2.set_title('Metrics Across Trials')
ax2.grid(True, alpha=0.3)

# Combine legends
lines = line1 + line2
labels = [l.get_label() for l in lines]
ax2.legend(lines, labels, loc='upper right')

# Plot 3: Response latency across trials
ax3 = axes[1, 0]
valid_latencies = [lat for lat in response_latencies if not np.isnan(lat)]
valid_trials = [i+1 for i, lat in enumerate(response_latencies) if not np.isnan(lat)]

ax3.plot(valid_trials, valid_latencies, 'o-', color='green', markersize=8, linewidth=2)
ax3.axhline(y=np.nanmean(response_latencies), color='green', linestyle='--', alpha=0.7, 
           label=f'Mean: {np.nanmean(response_latencies):.2f}s')
ax3.set_xlabel('Trial Number')
ax3.set_ylabel('Response Latency (s)')
ax3.set_title('Response Latency Across Trials')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Plot 4: Summary statistics
ax4 = axes[1, 1]
metrics = ['Baseline\n(mm)', 'Max Constr.\n(rel.)', 'Latency\n(s)']
means = [np.mean(baselines), np.mean(max_constrictions), np.nanmean(response_latencies)]
stds = [np.std(baselines), np.std(max_constrictions), np.nanstd(response_latencies)]

bars = ax4.bar(metrics, means, yerr=stds, capsize=5, alpha=0.7, color=['blue', 'red', 'green'])
ax4.set_title('Mean ± SD Across All Trials')
ax4.set_ylabel('Value')
ax4.grid(True, alpha=0.3)

# Add value labels on bars
for bar, mean, std in zip(bars, means, stds):
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height + std + 0.01,
             f'{mean:.3f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

# Step 9: Summary and Conclusions

Let's summarize what we've learned about pupillometry series analysis.

In [ ]:
print("=== PUPILLOMETRY SERIES ANALYSIS SUMMARY ===\n")

print("1. SERIES DATA CHARACTERISTICS:")
print(f"   ✓ Total recording duration: {series.get_time()[-1] - series.get_time()[0]:.1f} seconds")
print(f"   ✓ Number of stimuli: {len(pupil_measurements)}")
print(f"   ✓ Data points before preprocessing: {len(series.get_time())}")
print(f"   ✓ Data points after preprocessing: {len(series_final.get_time())}")
print(f"   ✓ Data retention rate: {100 * len(series_final.get_time()) / len(series.get_time()):.1f}%")

print(f"\n2. PREPROCESSING PIPELINE EFFECTIVENESS:")
print(f"   ✓ Artifact removal: Rate limiting < 2 mm/s")
print(f"   ✓ Noise reduction: {np.std(series.get_size()):.3f} → {np.std(series_final.get_size()):.3f} mm")
print(f"   ✓ Smoothing: 0.5s rolling window applied")
print(f"   ✓ Signal preservation: Key features maintained")

print(f"\n3. INDIVIDUAL MEASUREMENT EXTRACTION:")
print(f"   ✓ Successfully extracted {len(pupil_measurements)} individual responses")
print(f"   ✓ Pre-pulse duration: 10.0 seconds")
print(f"   ✓ Post-pulse duration: 30.0 seconds")
print(f"   ✓ All measurements include baseline and recovery periods")

print(f"\n4. CROSS-TRIAL CONSISTENCY:")
print(f"   • Baseline reliability (CV): {cv_baseline:.1f}%")
print(f"   • Max constriction reliability (CV): {cv_constriction:.1f}%")
print(f"   • Response latency reliability (CV): {cv_latency:.1f}%")

reliability_assessment = "Excellent" if cv_constriction < 10 else "Good" if cv_constriction < 20 else "Fair"
print(f"   ✓ Overall reliability: {reliability_assessment}")

print(f"\n5. COMPREHENSIVE FITTING RESULTS:")
print(f"   ✓ Successfully fitted {len(fits)} individual measurements")
print(f"   ✓ All phases analyzed: baseline, constriction, sustained, redilation")
print(f"   ✓ Model predictions generated for each trial")
print(f"   ✓ Parameter extraction completed")

print(f"\n6. PACKAGE CAPABILITIES DEMONSTRATED:")
print("   ✓ Continuous series data loading and preprocessing")
print("   ✓ Multi-stimulus series analysis")
print("   ✓ Automated measurement extraction")
print("   ✓ Batch processing and fitting")
print("   ✓ Cross-trial consistency analysis")
print("   ✓ Comprehensive visualization suite")
print("   ✓ Statistical reliability assessment")

print(f"\n=== SERIES ANALYSIS COMPLETE! ===\n")
print(f"This example demonstrated the complete pipeline for analyzing pupillometry series data.")
print(f"Perfect for experimental sessions with multiple trials and longitudinal studies! 🚀")

print(f"\nNext steps you might consider:")
print(f"• Export results to CSV/Excel for further statistical analysis")
print(f"• Compare series across different experimental conditions")
print(f"• Analyze habituation or adaptation effects across trials")
print(f"• Investigate individual differences in response patterns")